# Relationship 一对多

- 强烈推荐
- 优势在于: 能够自动完成一些创建关联表的工作


## 测试准备


### 定义 model


In [1]:
from typing import List, Optional

from sqlmodel import Field, Relationship, Session, SQLModel, create_engine


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    headquarters: str

    heroes: List["Hero"] = Relationship(back_populates="team")


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int] = Field(default=None, index=True)

    team_id: Optional[int] = Field(default=None, foreign_key="team.id")
    team: Optional[Team] = Relationship(back_populates="heroes")

### 创建表


In [2]:
sqlite_file_name = "relationship_one_to_many_database.db"
sqlite_url = f"sqlite:///{sqlite_file_name}"

engine = create_engine(sqlite_url, echo=False)
SQLModel.metadata.create_all(engine)

## 创建 Relationship


### 创建一对多的测试数据

- 先创建 team,再创建 hero
- 两个 team 并没有显式创建并 commit，而是自动被创建出来


In [3]:
with Session(engine) as session:
    team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
    team_z_force = Team(name="Z-Force", headquarters="Sister Margaret's Bar")

    hero_deadpond = Hero(name="Deadpond", secret_name="Dive Wilson", team=team_z_force)
    hero_rusty_man = Hero(
        name="Rusty-Man", secret_name="Tommy Sharp", age=48, team=team_preventers
    )
    hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
    session.add(hero_deadpond)
    session.add(hero_rusty_man)
    session.add(hero_spider_boy)
    session.commit()

    session.refresh(hero_deadpond)
    session.refresh(hero_rusty_man)
    session.refresh(hero_spider_boy)

    print("Created hero:", hero_deadpond)
    print("Created hero:", hero_rusty_man)
    print("Created hero:", hero_spider_boy)

Created hero: age=None id=1 name='Deadpond' secret_name='Dive Wilson' team_id=1
Created hero: age=48 id=2 name='Rusty-Man' secret_name='Tommy Sharp' team_id=2
Created hero: age=None id=3 name='Spider-Boy' secret_name='Pedro Parqueador' team_id=None


### 创建后，再分配关系

除了上述新建的时候，分配 team，还可以在后续给对象分配 team


In [5]:
with Session(engine) as session:
    hero_spider_boy.team = team_preventers
    session.add(hero_spider_boy)
    session.commit()
    session.refresh(hero_spider_boy)
    print("Updated hero:", hero_spider_boy)

Updated hero: age=None id=3 name='Spider-Boy' secret_name='Pedro Parqueador' team_id=2


### 创建多对一的数据

先创建多个 hero,再创建 team


In [6]:
with Session(engine) as session:
    hero_black_lion = Hero(name="Black Lion", secret_name="Trevor Challa", age=35)
    hero_sure_e = Hero(name="Princess Sure-E", secret_name="Sure-E")
    team_wakaland = Team(
        name="Wakaland",
        headquarters="Wakaland Capital City",
        heroes=[hero_black_lion, hero_sure_e],
    )
    session.add(team_wakaland)
    session.commit()
    session.refresh(team_wakaland)
    print("Team Wakaland:", team_wakaland)

Team Wakaland: id=3 name='Wakaland' headquarters='Wakaland Capital City'


## 查询


In [8]:
from sqlmodel import select

### 查询并 带出多关系

查询一对多数据的场景： 查询多，自动带出它的一


In [10]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.name == "Spider-Boy")
    result = session.exec(statement)
    hero_spider_boy = result.one()

    print("Spider-Boy's team again:", hero_spider_boy)
    print("Spider-Boy's team again:", hero_spider_boy.team)

Spider-Boy's team again: id=2 name='Preventers' headquarters='Sharp Tower'
Spider-Boy's team again: age=None id=3 name='Spider-Boy' secret_name='Pedro Parqueador' team_id=2


### 查询并 带出一关系

查询一对多数据的场景： 查询一，自动带出它的多


In [11]:
with Session(engine) as session:
    statement = select(Team).where(Team.name == "Preventers")
    result = session.exec(statement)
    team_preventers = result.one()
    for hero in team_preventers.heroes:
        print(hero)

age=48 id=2 name='Rusty-Man' secret_name='Tommy Sharp' team_id=2
age=None id=3 name='Spider-Boy' secret_name='Pedro Parqueador' team_id=2


## 移除
移除hero的team

In [12]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.name == "Spider-Boy")
    result = session.exec(statement)
    hero_spider_boy = result.one()

    hero_spider_boy.team = None
    session.add(hero_spider_boy)
    session.commit()

    session.refresh(hero_spider_boy)
    print("Spider-Boy without team:", hero_spider_boy)

Spider-Boy without team: age=None id=3 name='Spider-Boy' secret_name='Pedro Parqueador' team_id=None
